In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import datetime
import pickle
from shapely.geometry import Point

In [2]:
promise_zone = gpd.read_file('../data/MDHA_Promise_Zones/Export_Output_5.shp')
print(promise_zone.crs)
promise_zone.head(7 )

epsg:2274


,OBJECTID,ZONE_ID,SHAPE_STAr,SHAPE_STLe,geometry
0,1,1,1.053728e+08,49364.601403,"POLYGON ((1743366.812 678044.543, 1743367.698 ..."
1,2,2,7.670585e+07,45226.018917,"POLYGON ((1738690.545 661791.347, 1738733.644 ..."
2,3,3,2.985488e+08,75207.067487,"POLYGON ((1759602.544 668309.889, 1759640.155 ..."
3,4,4,2.713638e+08,74348.596054,"POLYGON ((1747799.003 654974.612, 1747821.293 ..."
4,5,5,3.928178e+08,106661.712102,"POLYGON ((1737013.744 692634.639, 1737122.605 ..."
5,6,6,9.535996e+07,59256.923574,"POLYGON ((1716935.654 667534.495, 1716992.481 ..."


In [3]:
promise_zone = promise_zone.to_crs(epsg = 4326)

In [4]:
polygon_zone_5 = promise_zone.loc[promise_zone['ZONE_ID']==5]

In [5]:
bus_stops = pd.read_csv('../data/busstops_cleaned.csv')
print(bus_stops.shape)

(2524, 5)


In [6]:
bus_stops['geometry'] = bus_stops.apply(lambda x: Point((x.lng,x.lat)),
                                        axis=1)

In [7]:
bus_geo = gpd.GeoDataFrame(bus_stops, 
                           crs = promise_zone.crs, 
                           geometry = bus_stops['geometry'])

In [8]:
stops_by_zone = gpd.sjoin( bus_geo,polygon_zone_5, op = 'within')

C:\Users\RMAUR\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
zone_5_center = polygon_zone_5.centroid

C:\Users\RMAUR\AppData\Local\Temp/ipykernel_6280/806582915.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zone_5_center = polygon_zone_5.centroid


In [10]:
stops_in_zone5 = stops_by_zone.loc[stops_by_zone['ZONE_ID']==5]

In [11]:
zone_5_area = (zone_5_center.y,zone_5_center.x)

In [12]:
# Import the file as a Pandas DataFrame
fp = '../data/july.csv'
july = pd.read_csv(fp)

In [13]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}
july.companyname = july.companyname.replace(company_dict)

In [14]:
july.pubdatetime = pd.to_datetime(july['pubdatetime'])

In [15]:
july.to_pickle("../data/july.pkl")

In [16]:
july_scooters = july[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]

In [17]:
july_1_scooters = july_scooters[july_scooters['pubdatetime']<='2019-07-02 00:00:00.000'] 

In [25]:
july_6_scooters = july_scooters[july_scooters['pubdatetime'].between('2019-07-06 00:00:00.000','2019-07-07 00:00:00.000')] 

In [29]:
july_6_scooters = july_6_scooters.set_index('pubdatetime')

In [22]:
july_1_scooters['geometry'] = july_1_scooters.apply(lambda x: Point(float(x.longitude), float(x.latitude)),axis=1)

In [32]:
july_6_scooters['geometry'] = july_6_scooters.apply(lambda x: Point(float(x.longitude), float(x.latitude)),axis=1)

In [36]:
jul1_scoots_geo = gpd.GeoDataFrame(july_1_scooters,crs=promise_zone.crs,geometry=july_1_scooters.geometry)
jul6_scoots_geo = gpd.GeoDataFrame(july_1_scooters,crs=promise_zone.crs,geometry=july_1_scooters.geometry)

In [34]:
jul1_scoots_by_zone = gpd.sjoin( jul1_scoots_geo,promise_zone, op = 'within')

C:\Users\RMAUR\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [37]:
jul6_scoots_by_zone = gpd.sjoin( jul1_scoots_geo,promise_zone, op = 'within')

C:\Users\RMAUR\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [42]:
jul1_scoots_zone5 = jul1_scoots_by_zone.loc[jul1_scoots_by_zone['ZONE_ID']==5]

In [41]:
jul6_scoots_zone5 = jul6_scoots_by_zone.loc[jul6_scoots_by_zone['ZONE_ID']==5]

In [47]:
# Create an empty list
jul1_hour_list = []

# Create a series with the different hours of the day
jul1_hours = pd.Series(jul1_scoots_zone5.index.hour.unique().sort_values())

# Create a list of points for each hour of the day
def create_list(hour):
    jul1_hour_list.append(jul1_scoots_zone5.loc[jul1_scoots_zone5.index.hour == hour,
                                ['latitude',
                                 'longitude']].
                        groupby(['latitude', 
                                 'longitude']).sum().reset_index().values.tolist())
jul1_hours.apply(create_list);

In [48]:
# Create an empty list
jul6_hour_list = []

# Create a series with the different hours of the day
jul6_hours = pd.Series(jul6_scoots_zone5.index.hour.unique().sort_values())

# Create a list of points for each hour of the day
def create_list(hour):
    jul6_hour_list.append(jul6_scoots_zone5.loc[jul6_scoots_zone5.index.hour == hour,
                                ['latitude',
                                 'longitude']].
                        groupby(['latitude', 
                                 'longitude']).sum().reset_index().values.tolist())
jul6_hours.apply(create_list);

In [53]:
from folium.plugins import HeatMapWithTime
# Add trip events to the map
ju6_zone5_map_time = folium.Map(location = zone_5_area, 
                      tiles="CartoDB Positron", 
                      zoom_start=13)


folium.GeoJson(polygon_zone_5).add_to(zone5_map_time)


HeatMapWithTime(df_hour_list, 
                auto_play=False, 
                max_opacity=0.5, 
                gradient = {0.2: '#FBD973', 
                            0.4: '#fa782f', 
                            0.75: '#F16578', 
                            1: '#782890'}).add_to(zone5_map_time)
zone5_map_time.save('../maps/zone5_map_time.html')
zone5_map_time

AttributeError: 'Map' object has no attribute 'm1'

In [52]:
help(folium.plugins.DualMap)

Help on class DualMap in module folium.plugins.dual_map:

class DualMap(folium.elements.JSCSSMixin, branca.element.MacroElement)
 |  DualMap(location=None, layout='horizontal', **kwargs)
 |  
 |  Create two maps in the same window.
 |  
 |  Adding children to this objects adds them to both maps. You can access
 |  the individual maps with `DualMap.m1` and `DualMap.m2`.
 |  
 |  Uses the Leaflet plugin Sync: https://github.com/jieter/Leaflet.Sync
 |  
 |  Parameters
 |  ----------
 |  location: tuple or list, optional
 |      Latitude and longitude of center point of the maps.
 |  layout : {'horizontal', 'vertical'}
 |      Select how the two maps should be positioned. Either horizontal (left
 |      and right) or vertical (top and bottom).
 |  **kwargs
 |      Keyword arguments are passed to the two Map objects.
 |  
 |  Examples
 |  --------
 |  >>> # DualMap accepts the same arguments as Map:
 |  >>> m = DualMap(location=(0, 0), tiles='cartodbpositron', zoom_start=5)
 |  >>> # Add th